In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
import os
import numpy as np

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

data_dir = "../../data/animals10"

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
model = models.alexnet(pretrained=False)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
model.to(device)

def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(init_weights)

for param in model.parameters():
    param.requires_grad = True

In [10]:
def evaluate(model, dataloader):
    model.eval()
    correct, total, total_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            _, pred = out.max(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total, total_loss / len(dataloader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4) 

epochs = 20
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_acc, _ = evaluate(model, train_loader)
    test_acc, test_loss = evaluate(model, test_loader)

    print(f"\n📊 [Epoch {epoch}]")
    print(f"   🔧 Train Accuracy : {train_acc*100:.2f}% | Train Loss: {train_loss:.4f}")
    print(f"   🧪 Test Accuracy  : {test_acc*100:.2f}% | Test Loss: {test_loss:.4f}")


📊 [Epoch 1]
   🔧 Train Accuracy : 43.93% | Train Loss: 1.8311
   🧪 Test Accuracy  : 43.87% | Test Loss: 1.6157

📊 [Epoch 2]
   🔧 Train Accuracy : 61.59% | Train Loss: 1.3529
   🧪 Test Accuracy  : 58.56% | Test Loss: 1.2105

📊 [Epoch 3]
   🔧 Train Accuracy : 69.37% | Train Loss: 1.1146
   🧪 Test Accuracy  : 64.66% | Test Loss: 1.0189

📊 [Epoch 4]
   🔧 Train Accuracy : 75.05% | Train Loss: 0.9565
   🧪 Test Accuracy  : 68.36% | Test Loss: 0.9224

📊 [Epoch 5]
   🔧 Train Accuracy : 78.79% | Train Loss: 0.8290
   🧪 Test Accuracy  : 71.27% | Test Loss: 0.8568

📊 [Epoch 6]
   🔧 Train Accuracy : 80.98% | Train Loss: 0.7084
   🧪 Test Accuracy  : 71.53% | Test Loss: 0.8647

📊 [Epoch 7]
   🔧 Train Accuracy : 82.20% | Train Loss: 0.6233
   🧪 Test Accuracy  : 70.31% | Test Loss: 0.8969

📊 [Epoch 8]
   🔧 Train Accuracy : 90.41% | Train Loss: 0.5314
   🧪 Test Accuracy  : 74.62% | Test Loss: 0.7647

📊 [Epoch 9]
   🔧 Train Accuracy : 88.21% | Train Loss: 0.4333
   🧪 Test Accuracy  : 72.43% | Test Loss: